# Bert fine-tunning in mnli ( original + negative sentences ) 





## preparation

In [ ]:
!nvidia-smi

Wed Aug 31 11:41:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install transformers
pip install datasets
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

## evaluate on bert_mnli

### mnli_datasets

In [ ]:
raw_datasets = load_dataset("glue", "mnli")

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

### model + tokenizer

In [ ]:
checkpoint = "vuiseng9/bert-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

NameError: ignored

### test

In [ ]:
raw_inputs = [
    "I love this.",
    "I like this so much!",
]
inputs = tokenizer(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2293, 2023, 1012,  102, 1045, 2066, 2023, 2061, 2172,  999,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
outputs = model(**inputs.to(model.device))

In [ ]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[ 3.5670,  0.4725, -4.7055]], device='cuda:0',
                                  grad_fn=<AddmmBackward0>))])

In [ ]:
print(outputs.logits.shape)

torch.Size([1, 3])


In [ ]:
label_map = model.config.id2label

In [ ]:
label_map[0]='entailment'
label_map[1]='neutral'
label_map[2]='contradiction'

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

entailment


### evaluate (original+ negative sentences)

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8495728234336859


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model.cuda() # gpu
model.eval()
with open('data/mnli_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model(**inputs.to(model.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7255


## fine-tunning

### datasets

In [ ]:
data_files = './data/mnli_completed_with_importance.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 2000
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [ ]:
label_map = {'contradiction':0 , 'neutral':1 , 'entailment':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 2000
})

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 140
    })
})

### prepare trainer evironment

In [ ]:
# download the checkpoint
checkpoint = "vuiseng9/bert-mnli"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

loading file https://huggingface.co/vuiseng9/bert-mnli/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/4fec1391a8a6385ab52caaeeb9dfbbb00e4bf73d0c9e76d8d552319792723cd1.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/vuiseng9/bert-mnli/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/9231718d3a685e917ca5fc1b54ef566c6516f141b957221a00cf0419d744492a.b1d3b19ab013240a348484166f1a44749cad6f108156f07e3784cbf6c2a27772
loading file https://huggingface.co/vuiseng9/bert-mnli/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/vuiseng9/bert-mnli/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/6ed4a883ab45e88581736447b6281c9f786754fe18f64ec05f1a665cd430ed16.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/vuiseng9/bert-mnli/resolve/main/tokenizer_config.json from c

In [ ]:
def tokenize_function(example):
    return tokenizer(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 140
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "bert-mnli-fine-tunned-neg-mnli", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mnli")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, hypothesis, Unnamed: 0, importance. If text, hypothesis, Unnamed: 0, importance are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1260
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 945


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.682108,0.750000
2,0.803400,0.819360,0.771429
3,0.803400,0.929380,0.792857


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, hypothesis, Unnamed: 0, importance. If text, hypothesis, Unnamed: 0, importance are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 140
  Batch size = 4
Saving model checkpoint to bert-mnli-fine-tunned-neg-mnli/checkpoint-500
Configuration saved in bert-mnli-fine-tunned-neg-mnli/checkpoint-500/config.json
Model weights saved in bert-mnli-fine-tunned-neg-mnli/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-mnli-fine-tunned-neg-mnli/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-mnli-fine-tunned-neg-mnli/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, hypothesis

TrainOutput(global_step=945, training_loss=0.5834153109757357, metrics={'train_runtime': 98.4348, 'train_samples_per_second': 38.401, 'train_steps_per_second': 9.6, 'total_flos': 86751371627928.0, 'train_loss': 0.5834153109757357, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "bert-mnli-fine-tunned-neg-mnli/checkpoint-500"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/MNLI/dev_mismatched.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[8], tokens[9], tokens[-1]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.7278275020341741

| Accuracy:  0.7278275020341741


In [ ]:
# evaluate on negation_benchmark snli
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('data/mnli_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:   0.838

| Accuracy:  0.838


### test on important and unimportante sentences

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 600
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.7433333333333333

| Accuracy:  0.7433333333333333


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.6574074074074074

| Accuracy:  0.6574074074074074


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.8189655172413793

| Accuracy:  0.8189655172413793


# Bert fine-tunning in snli ( original + negative sentences ) 





## preparation

In [ ]:
!nvidia-smi

Sun Aug 14 05:14:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install transformers
pip install datasets
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

## evaluate on bert_snli

### load the model and tokenizer

In [ ]:
# ERROR ! 

# checkpoint = "textattack/bert-base-uncased-snli"
# tokenizer_snli = AutoTokenizer.from_pretrained(checkpoint)
# model_snli = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

### fine-tunning bert-based-uncased on snli
the one fine-tunned in huggingface is not very performance, so I do it again

In [ ]:
datasets = load_dataset('snli')
datasets

In [ ]:
datasets = datasets.filter(lambda expression:expression['label']!=-1)

In [ ]:
print(set(datasets['train']['label']))

{0, 1, 2}


In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)
model

In [ ]:
def tokenize_function(example):
    return tokenizer(example["premise"], example["hypothesis"], truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9824
    })
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 549367
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9842
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "bert-based-uncased-snli", 
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    num_train_epochs = 3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load('glue', 'mnli')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 549367
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 137342


Epoch,Training Loss,Validation Loss,Accuracy
1,0.447500,0.430992,0.904186


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: hypothesis, premise. If hypothesis, premise are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9842
  Batch size = 4


Saving model checkpoint to bert-based-uncased-snli/checkpoint-137342
Configuration saved in bert-based-uncased-snli/checkpoint-137342/config.json
Model weights saved in bert-based-uncased-snli/checkpoint-137342/pytorch_model.bin
tokenizer config file saved in bert-based-uncased-snli/checkpoint-137342/tokenizer_config.json
Special tokens file saved in bert-based-uncased-snli/checkpoint-137342/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=137342, training_loss=0.4103601852901169, metrics={'train_runtime': 7671.9615, 'train_samples_per_second': 71.607, 'train_steps_per_second': 17.902, 'total_flos': 9734021181743922.0, 'train_loss': 0.4103601852901169, 'epoch': 1.0})

### reload the fine-tunned model : model_snli

In [ ]:
checkpoint = "bert-based-uncased-snli/checkpoint-137342"
tokenizer_snli = AutoTokenizer.from_pretrained(checkpoint)
model_snli = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

### test the label's order

In [ ]:
raw_inputs = [
    "Two women are embracing while holding to go packages.",
    "Two woman are holding packages.",
]
inputs = tokenizer_snli(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  2048,  2308,  2024, 23581,  2096,  3173,  2000,  2175, 14555,
          1012,   102,  2048,  2450,  2024,  3173, 14555,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
outputs = model_snli(**inputs.to(model_snli.device))

In [ ]:
outputs

SequenceClassifierOutput([('logits',
                           tensor([[ 5.2433, -1.1663, -2.9671]], grad_fn=<AddmmBackward0>))])

In [ ]:
print(outputs.logits.shape)

torch.Size([1, 3])


In [ ]:
label_map = model_snli.config.id2label
label_map

{0: 'LABEL_0', 1: 'LABEL_1', 2: 'LABEL_2'}

In [ ]:
label_map[0]='entailment'
label_map[1]='neutral'
label_map[2]='contradiction'

In [ ]:
label_map

{0: 'entailment', 1: 'neutral', 2: 'contradiction'}

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

entailment


### evaluate (original+ negative sentences)

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_snli.cuda() # gpu
model_snli.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer_snli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_snli(**inputs.to(model_snli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8899


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_snli.cuda() # gpu
model_snli.eval()
with open('data/snli_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_snli(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_snli(**inputs.to(model_snli.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.2745


## fine-tunning with negatives sentences

### datasets

In [ ]:
data_files = './data/snli_completed_with_importance.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4ddf541e06bd4d44/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 2000
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [ ]:
label_map = {'contradiction':0 , 'neutral':1 , 'entailment':2}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 2000
})

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 140
    })
})

### prepare trainer evironment

In [ ]:
# # download the checkpoint
# checkpoint = "textattack/bert-base-uncased-snli"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
def tokenize_function(example):
    return tokenizer_snli(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_snli)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 140
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "bert-snli-fine-tunned-neg-snli", 
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mnli")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_snli,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_snli,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: hypothesis, text, Unnamed: 0, importance. If hypothesis, text, Unnamed: 0, importance are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1260
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 945


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.607368,0.785714
2,0.571500,0.627607,0.835714
3,0.571500,0.677412,0.828571


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: hypothesis, text, Unnamed: 0, importance. If hypothesis, text, Unnamed: 0, importance are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 140
  Batch size = 4
Saving model checkpoint to bert-snli-fine-tunned-neg-snli/checkpoint-315
Configuration saved in bert-snli-fine-tunned-neg-snli/checkpoint-315/config.json
Model weights saved in bert-snli-fine-tunned-neg-snli/checkpoint-315/pytorch_model.bin
tokenizer config file saved in bert-snli-fine-tunned-neg-snli/checkpoint-315/tokenizer_config.json
Special tokens file saved in bert-snli-fine-tunned-neg-snli/checkpoint-315/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: hypothesis, text

TrainOutput(global_step=945, training_loss=0.4416000931351273, metrics={'train_runtime': 84.6771, 'train_samples_per_second': 44.64, 'train_steps_per_second': 11.16, 'total_flos': 60933367513368.0, 'train_loss': 0.4416000931351273, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "bert-snli-fine-tunned-neg-snli/checkpoint-945"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/SNLI/snli_1.0_dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[5], tokens[6], tokens[0]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        # print(prediction_label, target)
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.2966


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('data/snli_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.9165


### test on important and unimportante sentences

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 600
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.7433333333333333

| Accuracy:  0.83


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.6574074074074074

| Accuracy:  0.7807308970099668


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))
# | Accuracy:  0.8189655172413793

| Accuracy:  0.859375


# Bert fine-tunning in rte ( original + negative sentences ) 

## preparation

In [ ]:
!nvidia-smi

Sun Aug 14 09:19:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/memoire'

/content/drive/MyDrive/memoire


In [ ]:
%%bash
pip install datasets
pip install transformers
pip install deberta
pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=a4f223fa0839d23a708747b850682fe8a1e4a830b949272f1e4098e933cd7f66
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AdamW
import evaluate
from torch.utils.data import DataLoader

## evaluate debera_rte

In fact, there is one ready to use

### prepare the model and the tokenizer

In [ ]:
# download the checkpoint
checkpoint = "textattack/bert-base-uncased-RTE"
tokenizer_rte = AutoTokenizer.from_pretrained(checkpoint)
model_rte = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model_rte

### download datasets

In [ ]:
# download the data
datasets = load_dataset("glue", "rte")

Generating train split:   0%|          | 0/2490 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/277 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/rte/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# show the data structure
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 2490
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 277
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3000
    })
})

In [ ]:
print(set(datasets['train']['label']))

{0, 1}


### fine-tunning on rte

In [ ]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

tokenized_datasets = datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "deberta-based-large-rte", 
    evaluation_strategy="epoch",
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load('glue', 'rte')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2490
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1869


Epoch,Training Loss,Validation Loss,Accuracy
1,0.713600,0.693909,0.472924
2,0.713900,0.691656,0.527076
3,0.704500,0.650134,0.628159


The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 4


Saving model checkpoint to deberta-based-xlarge-rte/checkpoint-623
Configuration saved in deberta-based-xlarge-rte/checkpoint-623/config.json
Model weights saved in deberta-based-xlarge-rte/checkpoint-623/pytorch_model.bin
tokenizer config file saved in deberta-based-xlarge-rte/checkpoint-623/tokenizer_config.json
Special tokens file saved in deberta-based-xlarge-rte/checkpoint-623/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DebertaForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `DebertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 277
  Batch size = 4
Saving model checkpoint to deberta-based-xlarge-rte/checkpoint-1246
Configuration saved in deberta-based-xlarge-rte/checkpoint-1246/config.json
Model weights saved in deberta-based-xlarge-rte/checkpoint-1246/pytorch_

TrainOutput(global_step=1869, training_loss=0.7075068768235436, metrics={'train_runtime': 691.9515, 'train_samples_per_second': 10.796, 'train_steps_per_second': 2.701, 'total_flos': 1827335485635240.0, 'train_loss': 0.7075068768235436, 'epoch': 3.0})

### test

In [ ]:
raw_inputs = [
    "I love this.",
    "I like this so much!",
]
inputs = tokenizer_rte(raw_inputs[0], raw_inputs[1], padding=True, truncation=True, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101, 1045, 2293, 2023, 1012,  102, 1045, 2066, 2023, 2061, 2172,  999,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
outputs = model_rte(**inputs.to(model_rte.device))
outputs

SequenceClassifierOutput([('logits',
                           tensor([[ 1.5398, -0.7894]], grad_fn=<AddmmBackward0>))])

In [ ]:
print(outputs.logits.shape)

torch.Size([1, 2])


In [ ]:
label_map = model_rte.config.id2label
label_map

{0: 'LABEL_0', 1: 'LABEL_1'}

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
print(label_map[predictions])

LABEL_0


### evaluate on dev

In [ ]:
# # download the checkpoint
# checkpoint = "textattack/bert-base-uncased-RTE"
# tokenizer_rte = AutoTokenizer.from_pretrained(checkpoint)
# model_rte = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

All model checkpoint weights were used when initializing DebertaForSequenceClassification.

All the weights of DebertaForSequenceClassification were initialized from the model checkpoint at ./deberta-based-large-rte/checkpoint-1869.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DebertaForSequenceClassification for predictions without further training.


In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_rte.cuda() # gpu
model_rte.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_rte(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_rte(**inputs.to(model_rte.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7256317689530686


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_rte.cuda() # gpu
model_rte.eval()
with open('data/rte_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_rte(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_rte(**inputs.to(model_rte.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.6675


## Fine-tunning in a negative sentences dataset

### Download the datasets

In [ ]:
data_files = './data/rte_completed_with_importance.tsv'
neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
neg_datasets

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4776678d94aa2461/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'label', 'importance'],
    num_rows: 2000
})

In [ ]:
neg_datasets.features

{'Unnamed: 0': Value(dtype='int64', id=None),
 'hypothesis': Value(dtype='string', id=None),
 'importance': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None),
 'text': Value(dtype='string', id=None)}

In [ ]:
label_map = {'not_entailment':1, 'entailment':0}
labels = []
for label in neg_datasets['label']:
  labels.append(label_map[label])

In [ ]:
neg_datasets = neg_datasets.remove_columns('label')
neg_datasets = neg_datasets.add_column('labels', labels)

In [ ]:
neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 2000
})

In [ ]:
neg_datasets[1]

{'Unnamed: 0': 1,
 'hypothesis': 'Polio is under control in the world.',
 'importance': '0',
 'labels': 1,
 'text': "The most surprising news story of the past week must not be the 'wonderful' story that the AIDS epidemic in India is under control."}

In [ ]:
neg_datasets = neg_datasets.train_test_split(test_size=0.3)

In [ ]:
train_neg_datasets = neg_datasets['train'].train_test_split(test_size=0.1)

In [ ]:
train_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
        num_rows: 140
    })
})

### prepare trainer evironment

In [ ]:
# # download the checkpoint
# checkpoint = "textattack/bert-base-uncased-RTE"
# tokenizer_rte = AutoTokenizer.from_pretrained(checkpoint)
# model_rte = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
def tokenize_function(example):
    return tokenizer_rte(example["text"], example["hypothesis"], truncation=True)

tokenized_neg_datasets = train_neg_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer_rte)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_neg_datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1260
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 140
    })
})

In [ ]:
# most hyperparameters are set by default : like optimizer
training_args = TrainingArguments(
    "bert-rte-fine-tunned-neg-rte", 
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
)

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "rte")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 算出axis = -1 那个列的概率最大的那个
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model_rte,
    training_args,
    train_dataset=tokenized_neg_datasets["train"],
    eval_dataset=tokenized_neg_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer_rte,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, importance, hypothesis, text. If Unnamed: 0, importance, hypothesis, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1260
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 945


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.616852,0.807143
2,0.470200,0.999244,0.764286
3,0.470200,1.001556,0.785714


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, importance, hypothesis, text. If Unnamed: 0, importance, hypothesis, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 140
  Batch size = 4


Saving model checkpoint to bert-rte-fine-tunned-neg-rte/checkpoint-315
Configuration saved in bert-rte-fine-tunned-neg-rte/checkpoint-315/config.json
Model weights saved in bert-rte-fine-tunned-neg-rte/checkpoint-315/pytorch_model.bin
tokenizer config file saved in bert-rte-fine-tunned-neg-rte/checkpoint-315/tokenizer_config.json
Special tokens file saved in bert-rte-fine-tunned-neg-rte/checkpoint-315/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Unnamed: 0, importance, hypothesis, text. If Unnamed: 0, importance, hypothesis, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 140
  Batch size = 4
Saving model checkpoint to bert-rte-fine-tunned-neg-rte/checkpoint-630
Configuration saved in bert-rte-fine-tunned-neg-rte/checkpoint-630/config.json
Model weights saved in

TrainOutput(global_step=945, training_loss=0.3770102687613674, metrics={'train_runtime': 89.6406, 'train_samples_per_second': 42.168, 'train_steps_per_second': 10.542, 'total_flos': 104781922242000.0, 'train_loss': 0.3770102687613674, 'epoch': 3.0})

## Evaluate with the fine-tunned model

In [ ]:
# download the checkpoint
checkpoint = "bert-rte-fine-tunned-neg-rte/checkpoint-945"
tokenizer_neg = AutoTokenizer.from_pretrained(checkpoint)
model_neg = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Didn't find file bert-rte-fine-tunned-neg-rte/checkpoint-945/added_tokens.json. We won't load it.
loading file bert-rte-fine-tunned-neg-rte/checkpoint-945/vocab.txt
loading file bert-rte-fine-tunned-neg-rte/checkpoint-945/tokenizer.json
loading file None
loading file bert-rte-fine-tunned-neg-rte/checkpoint-945/special_tokens_map.json
loading file bert-rte-fine-tunned-neg-rte/checkpoint-945/tokenizer_config.json
loading configuration file bert-rte-fine-tunned-neg-rte/checkpoint-945/config.json
Model config BertConfig {
  "_name_or_path": "bert-rte-fine-tunned-neg-rte/checkpoint-945",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "finetuning_task": "glue:rte",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": 

In [ ]:
# evaluate on dev_mismatched
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('corpus/RTE/dev.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.6967509025270758


In [ ]:
# evaluate on negation_benchmark
label_map = {0: 'entailment', 1: 'not_entailment'}
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
with open('data/rte_completed_with_importance.tsv') as fin:
    fin.readline()
    for index, line in enumerate(fin):
        tokens = line.strip().split('\t')
        sent1, sent2, target = tokens[1], tokens[2], tokens[3]
        inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
        outputs = model_neg(**inputs.to(model_neg.device))
        prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
        prediction_label = label_map[prediction]
        ncorrect += int(prediction_label == target)
        nsamples += 1
print('| Accuracy: ', float(ncorrect)/float(nsamples))


| Accuracy:  0.9075


### test on important and unimportante sentences

In [ ]:
# # reload the dataset
# data_files = './data/mnli_completed_with_importance.tsv'
# neg_datasets = load_dataset('csv', data_files=data_files, sep='\t', split="train")
# neg_datasets

In [ ]:
test_neg_datasets = neg_datasets['test']

In [ ]:
test_neg_datasets_importance = test_neg_datasets.filter(lambda example: '1' in example['importance'])
test_neg_datasets_unimportance = test_neg_datasets.filter(lambda example: example['importance']=='0')

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
test_neg_datasets

Dataset({
    features: ['Unnamed: 0', 'text', 'hypothesis', 'importance', 'labels'],
    num_rows: 600
})

In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8133333333333334


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_importance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.7254098360655737


In [ ]:
# evaluate on negation_benchmark
ncorrect, nsamples = 0, 0
model_neg.cuda() # gpu
model_neg.eval()
for line in test_neg_datasets_unimportance:
    sent1, sent2, target = line['text'], line['hypothesis'], line['labels']
    inputs = tokenizer_neg(sent1, sent2, padding=True, truncation=True, return_tensors="pt") 
    outputs = model_neg(**inputs.to(model_neg.device))
    prediction = torch.nn.functional.softmax(outputs.logits, dim=-1).argmax().item()
    ncorrect += int(prediction == target)
    nsamples += 1
    
print('| Accuracy: ', float(ncorrect)/float(nsamples))

| Accuracy:  0.8885135135135135
